In [2]:
import pandas as pd
import numpy as np
import nltk
import os
import json
import requests 
import time
# import wrangle as w
import matplotlib.pyplot as plt
import unicodedata
from bs4 import BeautifulSoup
from typing import Dict, List, Optional, Union, cast
from env_miatta import github_token, github_username
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
%matplotlib inline
from time import strftime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# ACQUIRE

In [ ]:
# import json
# import pandas as pd

# def combine_json_files_and_save(json_file_names, output_file_name):
#     combined_data = []

#     for file_name in json_file_names:
#         with open(file_name, "r") as json_file:
#             data = json.load(json_file)
#             combined_data.extend(data)

#     with open(output_file_name, "w") as combined_json_file:
#         json.dump(combined_data, combined_json_file, indent=1)

# # List of JSON file names
# json_file_names = [
#     'data1.json',
#     'data3.json',
#     'data4.json',
#     # ... (other file names)
#     'data24.json'
# ]

# # Output file name for combined data
# output_file_name = "data2.json"

# # Call the function to combine JSON files and write the output
# combine_json_files_and_save(json_file_names, output_file_name)

# # Read the combined JSON data into a DataFrame
# words_df = pd.read_json(output_file_name)


DATA TYPE SUMMARY

26 integer data types originally now 3
9 object data type originally now 1
0 null values
no missing values

In [4]:
# Aquire data using methods described above.
words_df = pd.read_json('data2.json')
words_df



,repo,language,readme_contents
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...
3,learn-co-students/python-dictionaries-readme-d...,Jupyter Notebook,\n# Dictionaries \n\n### Introduction\n\nAfter...
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...
...,...,...,...
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
114,tastejs/todomvc,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
115,learn-co-students/jupyter-notebook-introductio...,Jupyter Notebook,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
116,hasura-imad/imad-2016-app,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."


In [5]:
# Dataset columns
words_df.columns

Index(['repo', 'language', 'readme_contents'], dtype='object')

In [6]:
words_df.language.value_counts()

JavaScript          31
Python              15
Java                11
HTML                 7
C                    6
Jupyter Notebook     5
Shell                5
Ruby                 4
C++                  3
CSS                  3
Go                   3
TypeScript           3
SCSS                 2
PowerShell           1
Kotlin               1
Bicep                1
HCL                  1
Vim script           1
Objective-C          1
Vue                  1
Name: language, dtype: int64

In [7]:
# To inspect the first few rows of the DataFrame, you can use the head functiotelco.head()
words_df.head().T


,0,1,2,3,4
repo,TheAlgorithms/Python,apache/flink,forezp/SpringCloudLearning,learn-co-students/python-dictionaries-readme-d...,angular/angular-phonecat
language,Python,Java,Java,Jupyter Notebook,JavaScript
readme_contents,"<div align=""center"">\n<!-- Title: -->\n <a hr...",# Apache Flink\n\nApache Flink is an open sour...,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...,\n# Dictionaries \n\n### Introduction\n\nAfter...,# AngularJS Phone Catalog Tutorial Application...


In [8]:
# Check for total of duplicates in data set 
words_df.duplicated().sum()

0

In [9]:
# column datatypes and missign values
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         105 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [10]:
# observed numerical values
words_df.describe()


,repo,language,readme_contents
count,118,105,118
unique,118,20,118
top,TheAlgorithms/Python,JavaScript,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
freq,1,31,1


In [11]:
words_df.shape


(118, 3)

In [12]:
# Check for missing values
missing_values = words_df.isnull()
missing_values


,repo,language,readme_contents
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
113,False,False,False
114,False,False,False
115,False,False,False
116,False,False,False


In [13]:
words_df.isnull().sum()


repo                0
language           13
readme_contents     0
dtype: int64

In [14]:
# Drop rows with any missing values
words_df_dropped = words_df.dropna()
words_df_dropped


,repo,language,readme_contents
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...
3,learn-co-students/python-dictionaries-readme-d...,Jupyter Notebook,\n# Dictionaries \n\n### Introduction\n\nAfter...
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...
...,...,...,...
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
114,tastejs/todomvc,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
115,learn-co-students/jupyter-notebook-introductio...,Jupyter Notebook,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
116,hasura-imad/imad-2016-app,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."


In [15]:
rows_with_missing_language = words_df[words_df['language'].isnull()]

# Change data types of the isolated rows to 'object'
rows_with_missing_language = rows_with_missing_language.astype({'language': 'object'})

# Display the modified isolated rows
rows_with_missing_language


,repo,language,readme_contents
9,vivienzou1/DL-Notes-for-interview,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
11,jtleek/datasharing,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
13,freefq/free,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
17,996icu/996.ICU,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
26,luchihoratiu/debug-via-ssh,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
29,mqyqingfeng/Blog,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
40,xitu/gold-miner,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
41,xingshaocheng/architect-awesome,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
54,dcxy/learngit,None,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
68,bloominstituteoftechnology/module-challenge-in...,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."


In [16]:
# column datatypes 
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         105 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [17]:
# Replace null values in the 'language' column with "Other"
words_df['language'].fillna("Other", inplace=True)

In [18]:
#column datatypes 
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         118 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [25]:
words_df

,repo,language,readme_contents
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...
3,learn-co-students/python-dictionaries-readme-d...,Jupyter Notebook,\n# Dictionaries \n\n### Introduction\n\nAfter...
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...
...,...,...,...
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
114,tastejs/todomvc,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
115,learn-co-students/jupyter-notebook-introductio...,Jupyter Notebook,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
116,hasura-imad/imad-2016-app,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."


# PREPARE

### Data Cleaning:
    
- Drop unnecessary axis
- Rename
- Find nulls
- Drop nulls
- Check preperation
- The data set has 4 columns and 1,470 rows
- Each row represents individual employee numerical data
- Each column is attributes of the employees

### TRAIN SPLIT

# EXPLORE

In [ ]:
def tokenize(text):
    """
    Tokenizes the words in the input string.
    """
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

def nlp_wrangle():
    """
    Performs data wrangling for natural language processing (NLP) tasks.
    Returns a processed DataFrame for NLP analysis.
    """
    # Load data from JSON file
    df = pd.read_json('data2.json')
    
    # Tokenize and clean contents
    df['clean_contents'] = df.readme_contents.apply(tokenize).apply(' '.join)
    df['clean_contents'] = df.clean_contents.apply(clean).apply(' '.join)
    
     # Words to remove
    words_to_remove = ['http', 'com', '124', 'www','github', 'top', 'go','107', '0','1','2','3','4', '5', '6', '7', '8','9', 'md','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'em', 'abbr', 'q','ins', 'del', 'dfn', 'kbd', 'pre', 'samp', 'var', 'br', 'div', 'a', 'img', 'param', 'ul','ol', 'li', 'dl', 'dt', 'dd']

    # Remove specific words from clean_contents
    for word in words_to_remove:
        df['clean_contents'] = df['clean_contents'].str.replace(word, '')

    # Add message_length and word_count columns
    df['message_length'] = df['clean_contents'].str.len()
    df['word_count'] = df.clean_contents.apply(clean).apply(len)

    # Keep only top languages and assign others to 'Other'
    languages_to_keep = ['JavaScript', 'Python', 'Java', 'TypeScript', 'HTML']
    df['language'] = np.where(df['language'].isin(languages_to_keep), df['language'], 'Other')

    # Filter DataFrame based on conditions
    df = df.loc[(df['word_count'] <= 10000) & (df['message_length'] <= 60000)]

    return df

In [ ]:
a= set(range(0,118)) - set(list(words_df_w.index))
# a = list(a)
a

In [ ]:
words_df_temp = words_df[words_df.index == 41]

In [ ]:
words_df_temp.readme_contents

In [ ]:
cleaned_list = []

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

for content in words_df.readme_contents:
    cleaned_text = clean(content)
    cleaned_list.append(cleaned_text)
c

In [ ]:
def split_data(df, variable):
    '''
    take in a DataFrame and target variable. return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    train_validate, test = train_test_split(df, test_size=.20, random_state=123, stratify=df[variable])
    
    train, validate = train_test_split(train_validate, 
                                       test_size=.25, 
                                       random_state=123,
                                      stratify = train_validate[variable])
    return train, validate, test

In [ ]:
cleaned_list


In [ ]:
se = pd.Series(cleaned_list)
words_df['cleaned_text'] = se.values


In [ ]:
words_df


In [ ]:
train, validate, test = split_data(df, 'language')
train.head()

In [ ]:
    separate specific words by language category

In [ ]:
# we can do that process with a join on a Series and not just a list
# we will do that all words and categories
# we will pass our basic cleaning on top of that

JavaScript_words = clean(' '.join(train[train.language=='JavaScript']['clean_contents']))
Python_words = clean(' '.join(train[train.language=='Python']['clean_contents']))
Java_words = clean(' '.join(train[train.language=='Java']['clean_contents']))
TypeScript_words = clean(' '.join(train[train.language=='TypeScript']['clean_contents']))
HTML_words = clean(' '.join(train[train.language=='HTML']['clean_contents']))
Other_words = clean(' '.join(train[train.language=='Other']['clean_contents']))
all_words = clean(' '.join(train['clean_contents']))

In [ ]:
JavaScript_words      

In [ ]:
Python_words              

In [ ]:
Java_words               

In [ ]:
HTML_words

In [ ]:
C_words

In [ ]:
Jupyter Notebook_words

In [ ]:
Shell_words

In [ ]:
Ruby_words
C++_words
CSS_words                  
Go_words
TypeScript           3
SCSS                 2
PowerShell           1
Kotlin               1
Bicep                1
HCL                  1
Vim script           1
Objective-C          1
Vue        